In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Federated/2/train/'

test_data_dir = 'D:/TQ/Federated/2/val/'


In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [5]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 192 samples
Arson: 136 samples
Fight: 240 samples
Normal: 199 samples
Riot: 253 samples
Traffic_Accident: 200 samples
Train_Accident: 176 samples


In [6]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [7]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model2 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [8]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 1119 images belonging to 7 classes.
Found 277 images belonging to 7 classes.
Found 351 images belonging to 7 classes.


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [10]:
history = model2.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

Epoch 1/5


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


140/140 [==============================] - 32s 208ms/step - loss: 0.9485 - accuracy: 0.6720 - val_loss: 0.4017 - val_accuracy: 0.8881
Epoch 2/5
140/140 [==============================] - 23s 161ms/step - loss: 0.2115 - accuracy: 0.9410 - val_loss: 0.4791 - val_accuracy: 0.8700
Epoch 3/5
140/140 [==============================] - 24s 173ms/step - loss: 0.1170 - accuracy: 0.9705 - val_loss: 0.4578 - val_accuracy: 0.8664
Epoch 4/5
140/140 [==============================] - 26s 188ms/step - loss: 0.0769 - accuracy: 0.9794 - val_loss: 0.5397 - val_accuracy: 0.8628
Epoch 5/5
140/140 [==============================] - 27s 192ms/step - loss: 0.0628 - accuracy: 0.9857 - val_loss: 0.4979 - val_accuracy: 0.8592


In [11]:
#model = tf.keras.models.load_model("Client2_Centralized.h5")

In [12]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model2.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

351/351 [==============================] - 10s 27ms/step


In [13]:
print("Accuracy:")
test_loss, test_accuracy = model2.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:
Test Loss: 0.09722714871168137
Test Accuracy: 0.9829059839248657


In [14]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[48  0  0  0  0  0  0]
 [ 0 35  0  0  0  0  0]
 [ 0  0 60  0  0  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  0  0 64  0  0]
 [ 0  0  0  0  0 50  0]
 [ 0  0  0  0  0  0 44]]


In [15]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix2.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [23]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    # True negatives are all the samples that are not our current GT class (not the current row) 
    # and were not predicted as the current class (not the current column)
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))
    
    # True positives are all the samples of our current GT class that were predicted as such
    true_positives = cm2[idx, idx]
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm2)
print(per_class_accuracies)

{'Animal_Abuse': 1.0, 'Arson': 1.0, 'Fight': 1.0, 'Normal': 0.9829059829059829, 'Riot': 1.0, 'Traffic_Accident': 0.98005698005698, 'Train_Accident': 1.0}


In [28]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm2[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm2[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 48, Total Instances: 48
Arson - True Positives: 35, Total Instances: 35
Fight - True Positives: 60, Total Instances: 60
Normal - True Positives: 45, Total Instances: 50
Riot - True Positives: 64, Total Instances: 64
Traffic_Accident - True Positives: 50, Total Instances: 50
Train_Accident - True Positives: 44, Total Instances: 44
{'Animal_Abuse': 1.0, 'Arson': 1.0, 'Fight': 1.0, 'Normal': 0.9, 'Riot': 1.0, 'Traffic_Accident': 1.0, 'Train_Accident': 1.0}


In [16]:
model2.save("Client2_Centralized.h5")

In [17]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      1.00      1.00        48
           Arson       1.00      1.00      1.00        35
           Fight       1.00      1.00      1.00        60
          Normal       1.00      0.90      0.95        50
            Riot       1.00      1.00      1.00        64
Traffic_Accident       0.91      1.00      0.95        50
  Train_Accident       1.00      1.00      1.00        44

        accuracy                           0.99       351
       macro avg       0.99      0.99      0.99       351
    weighted avg       0.99      0.99      0.99       351



In [19]:
client2 = tf.keras.models.load_model('client2_30_fedavg.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [20]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client2.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client2.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


351/351 [==============================] - 8s 22ms/step
Test Loss: 0.16063350439071655
Test Accuracy: 0.9715099930763245
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.98      0.99        48
           Arson       1.00      1.00      1.00        35
           Fight       1.00      1.00      1.00        60
          Normal       0.98      0.90      0.94        50
            Riot       1.00      1.00      1.00        64
Traffic_Accident       0.88      1.00      0.93        50
  Train_Accident       1.00      0.95      0.98        44

        accuracy                           0.98       351
       macro avg       0.98      0.98      0.98       351
    weighted avg       0.98      0.98      0.98       351



In [21]:
# Calculate the confusion matrix
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm)

Confusion Matrix Before Aggregation:
[[47  0  0  1  0  0  0]
 [ 0 35  0  0  0  0  0]
 [ 0  0 60  0  0  0  0]
 [ 0  0  0 45  0  5  0]
 [ 0  0  0  0 64  0  0]
 [ 0  0  0  0  0 50  0]
 [ 0  0  0  0  0  2 42]]


In [22]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix2f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [24]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    # True negatives are all the samples that are not our current GT class (not the current row) 
    # and were not predicted as the current class (not the current column)
    true_negatives = np.sum(np.delete(np.delete(cm, idx, axis=0), idx, axis=1))
    
    # True positives are all the samples of our current GT class that were predicted as such
    true_positives = cm[idx, idx]
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = (true_positives + true_negatives) / np.sum(cm)
print(per_class_accuracies)

{'Animal_Abuse': 0.9971509971509972, 'Arson': 1.0, 'Fight': 1.0, 'Normal': 0.9829059829059829, 'Riot': 1.0, 'Traffic_Accident': 0.98005698005698, 'Train_Accident': 0.9943019943019943}


In [26]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 47, Total Instances: 48
Arson - True Positives: 35, Total Instances: 35
Fight - True Positives: 60, Total Instances: 60
Normal - True Positives: 45, Total Instances: 50
Riot - True Positives: 64, Total Instances: 64
Traffic_Accident - True Positives: 50, Total Instances: 50
Train_Accident - True Positives: 42, Total Instances: 44
{'Animal_Abuse': 0.9791666666666666, 'Arson': 1.0, 'Fight': 1.0, 'Normal': 0.9, 'Riot': 1.0, 'Traffic_Accident': 1.0, 'Train_Accident': 0.9545454545454546}
